# MusicLM Experimentation
- Using Natural Language to Generate Song

In [2]:
import glob
import torch
# from torch.utils.tensorboard import SummaryWriter
# from musiclm_pytorch import MuLaN, AudioSpectrogramTransformer, TextTransformer
# from datetime import datetime
# # from midi2audio import FluidSynth

# import pandas as pd
# import numpy as np

In [6]:
torch.cuda.current_device()


AssertionError: Torch not compiled with CUDA enabled

In [3]:
device = torch.device("cuda")

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


# Load Sound Data
Load in MIDI data and convert it to WAV

In [15]:
# load in the metadata
# create a list of captions that concatenate the piece description and arousal
# lower case te caption list
midi_meta = pd.read_csv('../data/piano-labelled/labelled_piano_midi_metadata.csv')
midi_meta['caption_list'] = midi_meta['piece_description'].str.lower()+ ". " + midi_meta['piece_arousal'].str.lower()
midi_meta['wav_file'] = midi_meta['midi_file'].str.replace('.mid', '.wav')
midi_meta

,piece_id,piece_description,piece_arousal,piece_name,midi_file,caption_list,wav_file
0,0,very upbeat,Delighted,Lurking In The Darkness,Final Fantasy_PS1_Final Fantasy VII_Lurking In...,very upbeat. delighted,Final Fantasy_PS1_Final Fantasy VII_Lurking In...
1,1,I could tell the valence of the example was in...,Valence started out moderately negative and pr...,Lurking In The Darkness,Final Fantasy_PS1_Final Fantasy VII_Lurking In...,i could tell the valence of the example was in...,Final Fantasy_PS1_Final Fantasy VII_Lurking In...
2,2,For a second I thought this piece was going to...,This piece seemed to have a positive valence t...,Lurking In The Darkness,Final Fantasy_PS1_Final Fantasy VII_Lurking In...,for a second i thought this piece was going to...,Final Fantasy_PS1_Final Fantasy VII_Lurking In...
3,3,Bouncy and fun,Kind of sparatic,Lurking In The Darkness,Final Fantasy_PS1_Final Fantasy VII_Lurking In...,bouncy and fun. kind of sparatic,Final Fantasy_PS1_Final Fantasy VII_Lurking In...
4,4,nice,nice,Lurking In The Darkness,Final Fantasy_PS1_Final Fantasy VII_Lurking In...,nice. nice,Final Fantasy_PS1_Final Fantasy VII_Lurking In...
...,...,...,...,...,...,...,...
6100,6100,It started off slowly but happy and then built...,Seemed to remain consistent almost like it was...,One Winged Angel,Final Fantasy_PS1_Final Fantasy VII_One Winged...,it started off slowly but happy and then built...,Final Fantasy_PS1_Final Fantasy VII_One Winged...
6101,6101,This starts off a certain way then changes in ...,This is nostalgic because I recognize this and...,One Winged Angel,Final Fantasy_PS1_Final Fantasy VII_One Winged...,this starts off a certain way then changes in ...,Final Fantasy_PS1_Final Fantasy VII_One Winged...
6102,6102,The piece begins slow in tempo and then become...,"The beginning rhythm sounds suspenseful, makin...",One Winged Angel,Final Fantasy_PS1_Final Fantasy VII_One Winged...,the piece begins slow in tempo and then become...,Final Fantasy_PS1_Final Fantasy VII_One Winged...
6103,6103,started slow but picked up.,I feel it stayed the same,One Winged Angel,Final Fantasy_PS1_Final Fantasy VII_One Winged...,started slow but picked up. . i feel it stayed...,Final Fantasy_PS1_Final Fantasy VII_One Winged...


In [25]:
# prepare the dataset
# read in wav files as an array and associate it with the caption
# create a dataframe with the wav file and caption
# create a train and test split
# create a dataloader for the train and test split

import wave

def read_wav_as_np(wav_fname):
    """read in a wave file and convert it into a numpy array"""
    f = wave.open(wav_fname, 'rb')
    frames = f.getnframes()
    data = f.readframes(frames)
    data = np.frombuffer(data, dtype=np.int16)


    # Convert buffer to float32 using NumPy                                                                                 
    audio_as_np_int16 = np.frombuffer(data, dtype=np.int16)
    audio_as_np_float32 = audio_as_np_int16.astype(np.float32)
    
    # Normalise float32 array so that values are between -1.0 and +1.0                                                      
    max_int16 = 2**15
    audio_normalised = audio_as_np_float32 / max_int16
    return audio_normalised


dataset = []
data_path = "../data/piano-labelled/audio/"
for i, row in midi_meta.iterrows():
    piece = row['piece_name']
    caption = row['caption_list']
    
    for file in (glob.glob(f"{data_path}*.wav")):
        if piece in file:
            try:
                wav_file = file
                audio = read_wav_as_np(wav_file)
                dataset.append([audio, caption])
            except Exception as e:
                pass
dataset

../data/piano-labelled/audio\Legend_Of_Zelda_The_Ocarina_Of_Time_Ganondorf.wav
../data/piano-labelled/audio\Legend_Of_Zelda_The_Ocarina_Of_Time_Ganondorf.wav
../data/piano-labelled/audio\Legend_Of_Zelda_The_Ocarina_Of_Time_Ganondorf.wav
../data/piano-labelled/audio\Legend_Of_Zelda_The_Ocarina_Of_Time_Ganondorf.wav
../data/piano-labelled/audio\Legend_Of_Zelda_The_Ocarina_Of_Time_Ganondorf.wav
../data/piano-labelled/audio\Legend_Of_Zelda_The_Ocarina_Of_Time_Ganondorf.wav
../data/piano-labelled/audio\Legend_Of_Zelda_The_Ocarina_Of_Time_Ganondorf.wav
../data/piano-labelled/audio\Legend_Of_Zelda_The_Ocarina_Of_Time_Ganondorf.wav
../data/piano-labelled/audio\Legend_Of_Zelda_The_Ocarina_Of_Time_Ganondorf.wav
../data/piano-labelled/audio\Legend_Of_Zelda_The_Ocarina_Of_Time_Ganondorf.wav
../data/piano-labelled/audio\Legend_Of_Zelda_The_Ocarina_Of_Time_Ganondorf.wav
../data/piano-labelled/audio\Legend_Of_Zelda_The_Ocarina_Of_Time_Ganondorf.wav
../data/piano-labelled/audio\Legend_Of_Zelda_The_Oca

TypeError: 'in <string>' requires string as left operand, not float

# muLaN Playground
- Use to Experiment training MuLan. Goal is to train MuLaN on text NLP and the Wav sound file

In [7]:
# get a ton of <sound, text> pairs and train
wavs = torch.randn(100, 1024)
texts = torch.randint(0, 20000, (100, 256))

# split train and test 80/20
train_wavs = wavs[:80]
train_texts = texts[:80]
test_wavs = wavs[80:]
test_texts = texts[80:]

# wrap the dataset inside dataset loader
train_dataset = torch.utils.data.TensorDataset(train_wavs, train_texts)
test_dataset = torch.utils.data.TensorDataset(test_wavs, test_texts)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16)


In [8]:
# define model and loss function
audio_transformer = AudioSpectrogramTransformer(
    dim = 512,
    depth = 6,
    heads = 8,
    dim_head = 64,
    spec_n_fft = 128,
    spec_win_length = 24,
    spec_aug_stretch_factor = 0.8
)

text_transformer = TextTransformer(
    dim = 512,
    depth = 6,
    heads = 8,
    dim_head = 64
)

mulan = MuLaN(
    audio_transformer = audio_transformer,
    text_transformer = text_transformer
)

In [9]:
# Define a function to train on one epoch
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(train_loader):
        # Every data instance is an input + label pair
        wavs, texts = data

        loss = mulan(wavs, texts)
        loss.backward()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [10]:
# try training random stuff haha
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0
EPOCHS = 5

# set some large initial best loss to update over time
best_vloss = 1_000_000.

for epoch_number in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    mulan.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)

    
    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    mulan.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(test_loader):
            wavs, texts = vdata
            vloss = mulan(wavs, texts)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = '../models/mulan/model_{}_{}'.format(timestamp, epoch_number)
        torch.save(mulan.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:
spectrogram yielded shape of (65, 86), but had to be cropped to (64, 80) to be patchified for transformer
LOSS train 0.0 valid 1.9701570272445679
EPOCH 2:
LOSS train 0.0 valid 1.9701570272445679
EPOCH 3:
LOSS train 0.0 valid 1.9701570272445679
EPOCH 4:
LOSS train 0.0 valid 1.9701570272445679
EPOCH 5:
LOSS train 0.0 valid 1.9701570272445679


In [11]:
# after much training, you can embed sounds and text into a joint embedding space
# for conditioning the audio LM
embeds = mulan.get_audio_latents(wavs)  # durin training
embeds = mulan.get_text_latents(texts)  # during inference

# MuLAN Component

In [4]:
from musiclm_pytorch import MuLaNEmbedQuantizer

# setup the quantizer with the namespaced conditioning embeddings, unique per quantizer as well as namespace (per transformer)

quantizer = MuLaNEmbedQuantizer(
    mulan = mulan,                          # pass in trained mulan from above
    conditioning_dims = (1024, 1024, 1024), # say all three transformers have model dimensions of 1024
    namespaces = ('semantic', 'coarse', 'fine')
)

# now say you want the conditioning embeddings for semantic transformer

wavs = torch.randn(2, 1024)
conds = quantizer(wavs = wavs, namespace = 'semantic') # (2, 8, 1024) - 8 is number of quantizers

NameError: name 'mulan' is not defined

# AudioML Component
- Use to trian on the music piece only

In [50]:
import torch
from audiolm_pytorch import HubertWithKmeans
from audiolm_pytorch import SoundStream, SoundStreamTrainer
from audiolm_pytorch import SemanticTransformer, SemanticTransformerTrainer
from audiolm_pytorch import CoarseTransformer, CoarseTransformerTrainer
from audiolm_pytorch import FineTransformer, FineTransformerTrainer

"""
To train (or finetune) the three transformers that are a part of AudioLM, 
you simply follow the instructions over at audiolm-pytorch for training, 
but pass in the MulanEmbedQuantizer instance to the training classes under the keyword audio_conditioner
""" 

'\nTo train (or finetune) the three transformers that are a part of AudioLM, \nyou simply follow the instructions over at audiolm-pytorch for training, \nbut pass in the MulanEmbedQuantizer instance to the training classes under the keyword audio_conditioner\n'

# SoundStream Trainner

In [51]:

soundstream = SoundStream(
    codebook_size = 1024,
    rq_num_quantizers = 8,
    rq_groups = 2,                # this paper proposes using multi-headed residual vector quantization - https://arxiv.org/abs/2305.02765
    attn_window_size = 128,       # local attention receptive field at bottleneck
    attn_depth = 2                # 2 local attention transformer blocks - the soundstream folks were not experts with attention, so i took the liberty to add some. encodec went with lstms, but attention should be better
)

trainer = SoundStreamTrainer(
    soundstream,
    folder ='../data/piano-labelled/audio/',
    batch_size = 4,
    grad_accum_every = 8,         # effective batch size of 32
    data_max_length_seconds = 2,  # train on 2 second audio
    num_train_steps = 1_000_000
).cuda()

trainer.train()

# after a lot of training, you can test the autoencoding as so

audio = torch.randn(10080).cuda()
recons = soundstream(audio, return_recons_only = True) # (1, 10080) - 1 channel

training with dataset of 94 samples and validating with randomly splitted 5 samples


AssertionError: Torch not compiled with CUDA enabled

# Semantic Transformer

In [52]:
# load in the pretrained wav2vec model and kmeans codebook
wav2vec = HubertWithKmeans(
    checkpoint_path = '../models/musicLM/pretrained_checkpoints/hubert_base_ls960.pt',
    kmeans_path = '../models/musicLM/pretrained_checkpoints/hubert_base_ls960_L9_km500.bin'
)


semantic_transformer = SemanticTransformer(
    num_semantic_tokens = wav2vec.codebook_size,
    dim = 1024,
    depth = 6,
    flash_attn = True
).cuda()


trainer = SemanticTransformerTrainer(
    transformer = semantic_transformer,
    wav2vec = wav2vec,
    audio_conditioner = quantizer,   # pass in the MulanEmbedQuantizer instance above
    folder ='../data/piano-labelled/audio/',
    batch_size = 1,
    data_max_length = 320 * 32,
    num_train_steps = 1
)

trainer.train()

AssertionError: Torch not compiled with CUDA enabled

# Coarse Transformer

In [ ]:
soundstream = SoundStream.init_and_load_from('/path/to/trained/soundstream.pt')

coarse_transformer = CoarseTransformer(
    num_semantic_tokens = wav2vec.codebook_size,
    codebook_size = 1024,
    num_coarse_quantizers = 3,
    dim = 512,
    depth = 6,
    flash_attn = True
)

trainer = CoarseTransformerTrainer(
    transformer = coarse_transformer,
    codec = soundstream,
    wav2vec = wav2vec,
    audio_conditioner = quantizer,   # pass in the MulanEmbedQuantizer instance above
    folder ='../data/piano-labelled/audio/',
    batch_size = 1,
    data_max_length = 320 * 32,
    num_train_steps = 1_000_000
)

trainer.train()

# Fine Transformer

In [ ]:

soundstream = SoundStream.init_and_load_from('/path/to/trained/soundstream.pt')

fine_transformer = FineTransformer(
    num_coarse_quantizers = 3,
    num_fine_quantizers = 5,
    codebook_size = 1024,
    dim = 512,
    depth = 6,
    flash_attn = True
)

trainer = FineTransformerTrainer(
    transformer = fine_transformer,
    codec = soundstream,
    audio_conditioner = quantizer,   # pass in the MulanEmbedQuantizer instance above
    folder ='../data/piano-labelled/audio/',
    batch_size = 1,
    data_max_length = 320 * 32,
    num_train_steps = 1_000_000
)

trainer.train()

# Instantiate audoML Model Component

In [ ]:
from audiolm_pytorch import AudioLM

audio_lm = AudioLM(
    wav2vec = wav2vec,
    codec = soundstream,
    semantic_transformer = semantic_transformer,
    coarse_transformer = coarse_transformer,
    fine_transformer = fine_transformer
)


# Instantiate musicLM Model Component

In [33]:
# you need the trained AudioLM (audio_lm) from above
# with the MulanEmbedQuantizer (mulan_embed_quantizer)

from musiclm_pytorch import MusicLM

musiclm = MusicLM(
    audio_lm = audio_lm,                 # `AudioLM` from https://github.com/lucidrains/audiolm-pytorch
    mulan_embed_quantizer = quantizer    # the `MuLaNEmbedQuantizer` from above
)

music = musiclm('the crystalline sounds of the piano in a ballroom', num_samples = 4) # sample 4 and pick the top match with mulan

NameError: name 'audio_lm' is not defined